In [1]:
!pip install --upgrade google-cloud-aiplatform

In [3]:
import socket
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="us-central1"

UNIQUE_PREFIX = socket.gethostname()
UNIQUE_PREFIX = re.sub('[^A-Za-z0-9]+', '', UNIQUE_PREFIX)

BUCKET_NAME = f"{PROJECT_ID}-{UNIQUE_PREFIX}-{LOCATION}"

BUCKET_URI = f"gs://{BUCKET_NAME}"  # @param {type:"string"}

! gcloud config set project $PROJECT_ID
! gcloud storage buckets create {BUCKET_URI} --project={PROJECT_ID} --location={LOCATION}
! mkdir output

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

Creating gs://cloud-llm-preview1-soratest-us-central1/...


In [8]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Tool,
    grounding,
)

# TODO(developer): Update and un-comment below line
# project_id = "PROJECT_ID"

vertexai.init(project=PROJECT_ID, location="us-central1")

model = GenerativeModel(model_name="gemini-1.5-pro-001")

# Use Google Search for grounding
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

prompt = "When is the next total solar eclipse in US?"
response = model.generate_content(
    prompt,
    tools=[tool],
    generation_config=GenerationConfig(
        temperature=0.0,
        # random_seed = 1,
    ),
)

print(response)

candidates {
  content {
    role: "model"
    parts {
      text: "The next total solar eclipse in the US will occur on August 23, 2044. \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.16478510200977325
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.05997643992304802
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.4131810963153839
    severity: HARM_SEVERITY_LOW
    severity_score: 0.3159469962120056
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.21650810539722443
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.15304216742515564
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.15102779865264893
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_sco

In [5]:

import vertexai
from vertexai.generative_models import GenerativeModel, Part

# TODO(developer): Update and un-comment below lines
# project_id = "PROJECT_ID"

vertexai.init(project=PROJECT_ID, location="us-central1")

model = GenerativeModel(model_name="gemini-1.5-flash-001")

prompt = """
Provide a description of the video.
The description should also contain anything important which people say in the video.
"""

video_file_uri = "gs://cloud-samples-data/generative-ai/video/pixel8.mp4"
video_file = Part.from_uri(video_file_uri, mime_type="video/mp4")

contents = [video_file, prompt]

response = model.generate_content(contents)
print(response.text)

The video opens with a shot of a train passing by in a city at night. The camera then pans down to a woman walking along a street. She is a photographer in Tokyo. She says that Tokyo has many faces and that the city at night is totally different from what you see during the day. She then introduces the feature "Video Boost" from the new Pixel phone and says that it activates "Night Sight" in low light to make the quality even better.  The woman then visits Sancha, where she used to live when she first moved to Tokyo. She says that she has a lot of great memories there.  She is pleased with the video quality on her new Pixel.  The woman then moves to Shibuya and points out a building. The video concludes with a shot of the city at night. 


In [8]:
!pip install --upgrade google-cloud-aiplatform
# gcloud auth application-default login


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [9]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

textsi_1 = "" 

def generate():
  vertexai.init(project="my-project-0004-346516", location="us-central1")
  model = GenerativeModel(
    "gemini-1.5-pro-001",
    system_instruction=[textsi_1]
  )
  responses = model.generate_content(
      [document1, text1],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
  )

  for response in responses:
    print(response.text, end="")

document1 = Part.from_uri(
    mime_type="application/pdf",
    uri="gs://cloud-samples-data/generative-ai/pdf/earnings_statement.pdf")
text1 = """You are a document entity extraction specialist. Given a document, your task is to extract the text value of the following entities:
{
 \"earning_item\": [
  {
   \"earning_rate\": \"\",
   \"earning_hours\": \"\",
   \"earning_type\": \"\",
   \"earning_this_period\": \"\"
  }
 ],
 \"direct_deposit_item\": [
  {
   \"direct_deposit\": \"\",
   \"employee_account_number\": \"\"
  }
 ],
 \"current_deduction\": \"\",
 \"ytd_deduction\": \"\",
 \"employee_id\": \"\",
 \"employee_name\": \"\",
 \"employer_name\": \"\",
 \"employer_address\": \"\",
 \"federal_additional_tax\": \"\",
 \"federal_allowance\": \"\",
 \"federal_marital_status\": \"\",
 \"gross_earnings\": \"\",
 \"gross_earnings_ytd\": \"\",
 \"net_pay\": \"\",
 \"net_pay_ytd\": \"\",
 \"ssn\": \"\",
 \"pay_date\": \"\",
 \"pay_period_end\": \"\",
 \"pay_period_start\": \"\",
 \"state_additional_tax\": \"\",
 \"state_allowance\": \"\",
 \"state_marital_status\": \"\",
 \"tax_item\": [
  {
   \"tax_this_period\": \"\",
   \"tax_type\": \"\",
   \"tax_ytd\": \"\"
  }
 ]
}

- The JSON schema must be followed during the extraction.
- The values must only include text strings found in the document.
- Generate null for missing entities."""

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

generate()



NameError: name 'textsi_1' is not defined